In [2]:
import ctypes
from ctypes import c_char_p, c_int
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import os
import io
import contextlib
import subprocess
import threading
import queue

CALLBACK_TYPE = ctypes.CFUNCTYPE(None, ctypes.c_char_p)

@CALLBACK_TYPE
def status_callback(message):

    print("Status Update:", message.decode('utf-8'))
    current_tile = 0
    if "Number of tiles" in message.decode('utf-8'):
        app.num_tiles = int(message.decode('utf-8').split()[3].strip())
    else:
        current_tile = int(message.decode('utf-8').split()[3].strip())
        app.processedTiles += 1
        progress = 100 / app.num_tiles
        app.increase_progress(progress)
    
    #reseting the progress bar
    if (app.processedTiles) == app.num_tiles:
        show_message("Done")
        app.reset_progress()
        
    #Checking if at least one tile has been processed, meaning we can safely delete the original image as it has been loaded
    if "Encrypted" in message.decode('utf-8') and current_tile == app.num_tiles-1:
        app.delete_original_image()

@CALLBACK_TYPE
def status_callback_scramble_image(message):
    print("Status Update:", message.decode('utf-8'))

def show_message(message):
    if app:
        messagebox.showinfo("Sparse Encryption", message)
    
class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tooltip_window = None
        self.widget.bind("<Enter>", self.show_tooltip)
        self.widget.bind("<Leave>", self.hide_tooltip)

    def show_tooltip(self, event=None):
        x = self.widget.winfo_rootx() + 20
        y = self.widget.winfo_rooty() + 20
        self.tooltip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)
        tw.wm_geometry(f"+{x}+{y}")
        label = tk.Label(tw, text=self.text, background="white", relief="solid", borderwidth=1)
        label.pack()

    def hide_tooltip(self, event=None):
        if self.tooltip_window:
            self.tooltip_window.destroy()
            self.tooltip_window = None

class SimpleUIApp:
    def __init__(self, root):
        
        self.current_directory = os.getcwd()
        self.file_path = ""
        self.folder_path = ""
        self.dll_path = self.current_directory +"\DLL_Test_1.dll"
        self.num_tiles = 0
        self.tile_size = 128
        self.processedTiles = 0
        self.iterations = 300
        self.removeNoise = tk.BooleanVar()
        self.threads = 3
        self.acceleration = 3
        self.messege ="Done"
        self.root = root
        self.root.title("Sparse Encryption")
        self.root.geometry("380x470")
        
        self.entry_iterations_label = tk.Label(root, text="Number of iterations:")
        self.entry_iterations_label.pack()
        self.entry_iterations_frame = tk.Frame(root)
        self.entry_iterations_frame.pack()
        self.entry_iterations = tk.Entry(self.entry_iterations_frame)
        self.entry_iterations.pack(side=tk.LEFT)
        self.entry_iterations_button = tk.Button(self.entry_iterations_frame, text="Save", command=self.save_iterations)
        self.entry_iterations_button.pack(side=tk.LEFT)
        ToolTip(self.entry_iterations, "Number of iterations for the decryption algorithm, 300 seems to be optimal fo tile size = 128, " 
                             "larger tile sizes may require less iterations")

        self.entry_threads_label = tk.Label(root, text="Number of threads:")
        self.entry_threads_label.pack()
        self.entry_threads_frame = tk.Frame(root)
        self.entry_threads_frame.pack()
        self.entry_threads = tk.Entry(self.entry_threads_frame)
        self.entry_threads.pack(side=tk.LEFT)
        self.entry_threads_button = tk.Button(self.entry_threads_frame, text="Save", command=self.save_threads)
        self.entry_threads_button.pack(side=tk.LEFT)
        ToolTip(self.entry_threads, "Number of separate threads used for encryption/decryption, more threads should equal better speed"
                             "but not always depending on hardware")

        self.entry_tile_size_label = tk.Label(root, text="Tile size:")
        self.entry_tile_size_label.pack()
        self.entry_tile_size_frame = tk.Frame(root)
        self.entry_tile_size_frame.pack()
        self.entry_tile_size = tk.Entry(self.entry_tile_size_frame)
        self.entry_tile_size.pack(side=tk.LEFT)
        self.entry_tile_size_button = tk.Button(self.entry_tile_size_frame, text="Save", command=self.save_tile_size)
        self.entry_tile_size_button.pack(side=tk.LEFT)
        ToolTip(self.entry_tile_size, "Smaller tile sizes require less memory while larger ones may produce better quality at the cost of performance")
        
        self.checkbox_noise = tk.Checkbutton(root, text="Remove noise", variable=self.removeNoise)
        self.checkbox_noise.pack()
        ToolTip(self.checkbox_noise, "Noise reduction for the decrypted image, may improve quality for smaller tile sizes")

        self.delete_original_var = tk.IntVar()
        self.delete_original = tk.Checkbutton(root, text="Delete original image", variable=self.delete_original_var, command=self.toggle_subcheckbox)
        self.delete_original.pack()
        ToolTip(self.delete_original, "WARNING : This will delete the original image")

        self.subcheckbox_var = tk.IntVar()
        self.subcheckbox = tk.Checkbutton(root, text="Scramble image first", variable=self.subcheckbox_var)
        self.subcheckbox.pack_forget()  # Hide the subcheckbox initially
        ToolTip(self.subcheckbox, "WARNING : This will scramble the original image"
                " so that recovery cannot be possible even after deletion")

        self.dropdown_label = tk.Label(root, text="Compute acceleration:")
        self.dropdown_label.pack()
        self.dropdown_var = tk.StringVar()
        self.dropdown = ttk.Combobox(root, textvariable=self.dropdown_var)
        self.dropdown['values'] = ("CPU Only", "GPU Only", "Hybrid Acceleration")  # Set initial options
        self.dropdown.current(2)
        self.dropdown.pack()
        ToolTip(self.dropdown, "Select an option from the dropdown")

        self.encrypt_button = tk.Button(root, text="Encrypt", command=self.encrypt_file)
        self.encrypt_button.pack()
        ToolTip(self.encrypt_button, "Click to encrypt the selected image")

        self.decrypt_button = tk.Button(root, text="Decrypt", command=self.decrypt_file)
        self.decrypt_button.pack()
        ToolTip(self.decrypt_button, "Click to decrypt the selected image")
        
        self.browse_button = tk.Button(root, text="Browse File", command=self.browse_file)
        self.browse_button.pack()
        ToolTip(self.browse_button, "Click to browse for a file")

        self.select_folder_button = tk.Button(root, text="Select Folder", command=self.select_folder)
        self.select_folder_button.pack()
        ToolTip(self.select_folder_button, "Click to select a folder")

        self.progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
        self.progress_bar.pack(pady=20)
        
        self.progress_bar["value"] = 0
        self.progress_bar["maximum"] = 100

        self.status_label = tk.Label(root, text="Status: Waiting to start...")
        self.status_label.pack(pady=20)

        self.entry1_value = ""
        self.entry2_value = ""
    
    def increase_progress(self, value):
        self.progress_bar['value'] = self.progress_bar['value'] + value
    
    def set_progress(self, value):
        self.progress_bar['value'] = value
        
    def reset_progress(self):
        self.progress_bar['value'] = 0
        self.status_label.config(text="Status: Waiting to start...")
        self.processedTiles = 0
        self.encrypt_button.config(state='normal')
        self.decrypt_button.config(state='normal')

    def delete_original_image(self):
        dll = ctypes.CDLL(self.dll_path)
        input_path = self.file_path
        input_path_c = input_path.encode('utf-8')

        if(self.delete_original_var.get()):
            dll_function_scramble = getattr(dll, "deleteOriginalImage")
            task_thread_scramble = threading.Thread(target=dll_function_scramble, args=(status_callback_scramble_image, input_path_c, self.subcheckbox_var.get()))
            task_thread_scramble.start()
        
    def encrypt_image_dll(self):
        
        if self.file_path == "" or self.folder_path == "":
            show_message("Error : Select a valid file path and destination folder")
            return -1
        
        match self.dropdown_var.get():
            case "CPU Only":
                self.acceleration = 3
            case "GPU Only":
                self.acceleration = 2
            case "Hybrid Acceleration":
                self.acceleration = 1
                
        dll = ctypes.CDLL(self.dll_path)
        dll_function = getattr(dll, "encryptAndWriteFile")
        
        input_path = self.file_path
        input_path_c = input_path.encode('utf-8')
        
        output_path = self.folder_path
        file_name = os.path.basename(input_path)
        
        if file_name.split(".")[1] in {"png","jpg","jpeg"}:
            print("Correct file extention " + file_name.split(".")[1])
        else:
            show_message("Error: Incorrect file extention, can only encrypt images that are : .png, .pjg or .jpeg ")
            return -1
        
        output_path = output_path + "/" + file_name.replace(".png",".se")
        output_path_c = output_path.encode('utf-8')
        
        print(input_path)
        print(output_path)
        print(self.tile_size)
        passphrase = "4358fv53829f23"
        passphrase_c = passphrase.encode('utf-8')
        
        dll_function.argtypes = [CALLBACK_TYPE, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_int, ctypes.c_int, ctypes.c_int]
        dll_function.restype = None
        
        task_thread = threading.Thread(target=dll_function, args=(status_callback, input_path_c, output_path_c, passphrase_c, self.tile_size, self.acceleration, self.threads))
        task_thread.start()
        self.messege = "Finished encrypting image"
        self.status_label.config(text="Status: Encrypting...")
        
        self.encrypt_button.config(state='disabled')
        self.decrypt_button.config(state='disabled')
            
    def decrypt_image_dll(self):
        
        if self.file_path == "" or self.folder_path == "":
            show_message("Error: Select a valid file path and destination folder")
            return -1
        
        match self.dropdown_var.get():
            case "CPU Only":
                self.acceleration = 3
            case "GPU Only":
                self.acceleration = 2
            case "Hybrid Acceleration":
                self.acceleration = 1
                
        print(self.removeNoise.get())
        dll = ctypes.CDLL(self.dll_path)
        dll_function = getattr(dll, "decryptAndWriteFile")
        
        input_path = self.file_path
        input_path_c = input_path.encode('utf-8')
        
        output_path = self.folder_path
        file_name = os.path.basename(input_path)
        
        if file_name.split(".")[1] in {"se"}:
            print("Correct file extention " + file_name.split(".")[1])
        else:
            show_message("Error: Incorrect file extention, can only decrypt images encrypted using this app with the extention .se")
            return -1
        
        output_path = output_path + "/" + file_name.replace(".se","_decrypted.png")
        output_path_c = output_path.encode('utf-8')
        
        print(input_path)
        print(output_path)
        passphrase = "4358fv53829f23"
        passphrase_c = passphrase.encode('utf-8')
        
        dll_function.argtypes = [CALLBACK_TYPE, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p, ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int]
        dll_function.restype = None
        
        task_thread = threading.Thread(target=dll_function, args=(status_callback, input_path_c, output_path_c, passphrase_c, self.acceleration, self.threads, self.iterations, int(self.removeNoise.get())))
        task_thread.start()
        self.messege = "Finished decrypting image"
        self.status_label.config(text="Status: Decrypting...")
        
        self.encrypt_button.config(state='disabled')
        self.decrypt_button.config(state='disabled')
    
    def save_iterations(self):
        self.entry_iterations_value = self.entry_iterations.get()
        self.iteration = int(self.entry_iterations_value)
        print(f"Iterations saved: {self.iteration}")

    def save_threads(self):
        self.entry_threads_value = self.entry_threads.get()
        self.threads = int(self.entry_threads_value)
        print(f"Threads saved: {self.threads}")
        
    def save_tile_size(self):
        self.entry_tile_size_value = self.entry_tile_size.get()
        self.tile_size = int(self.entry_tile_size_value)
        print(f"Tile size saved: {self.tile_size}")

    def browse_file(self):
        self.file_path = filedialog.askopenfilename()
    
    def encrypt_file(self):
        self.encrypt_image_dll()
        
    def decrypt_file(self):
        self.decrypt_image_dll()

    def select_folder(self):
        self.folder_path = filedialog.askdirectory()

    def toggle_subcheckbox(self):
        if self.delete_original_var.get():
            self.subcheckbox.pack(after=self.delete_original)
        else:
            self.subcheckbox.pack_forget()
        
app = None

if __name__ == "__main__":
    root = tk.Tk()
    app = SimpleUIApp(root)
    root.mainloop()


